In [1]:
import os
import numpy as np
from ResNet import resnet
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
import tensorflow as tf
from tensorflow.keras.optimizers import SGD

In [2]:
def show_results(log):
  val_loss = log.history['val_loss']
  val_acc = log.history['val_accuracy']
      
  fig, axes = plt.subplots(1, 2, figsize=(14,4))
  ax1, ax2 = axes
  ax1.plot(log.history['loss'], label='train')
  ax1.plot(val_loss, label='test')
  ax1.set_xlabel('epoch'); ax1.set_ylabel('loss')
  ax2.plot(log.history['accuracy'], label='train')
  ax2.plot(val_acc, label='test')
  ax2.set_xlabel('epoch'); ax2.set_ylabel('accuracy')
  for ax in axes: ax.legend()

In [3]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_path = "models/model"

monitor = "val_accuracy"

mode = "max"

model_checkpoint = ModelCheckpoint(checkpoint_path, monitor=monitor, verbose=1, save_best_only=True, save_weights_only=True, mode=mode)

In [4]:
path = os.path.abspath(os.getcwd())+'\\resultados\\'

# Guardamos la configuracion del NumPy np.load
np_load_old = np.load

# Modificamos un parametro del np.load por defecto que no permitia realizar la descarga
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

train_data = np.load(path+'train_data.npy')
train_labels = np.load(path+'train_labels.npy')
test_data = np.load(path+'test_data.npy')
test_labels = np.load(path+'test_labels.npy')
labels = np.load(path+'labels.npy')

# Restauramo np.load para su futuro uso regular
np.load = np_load_old

In [5]:
bs = 128
nb_classes = 200
nb_epoch = 20

# input image dimensions
img_rows, img_cols = 64, 64
# The images are RGB
img_channels = 3

input_shape = (img_rows, img_cols, img_channels)

# The data, shuffled and split between train and test sets:
X_train = train_data
Y_train = train_labels
X_test = test_data
Y_test = test_labels
del train_data
del train_labels
del test_data
del test_labels
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# subtract mean and normalize
mean_image = np.mean(X_train, axis=0)
X_train -= mean_image
X_test -= mean_image
X_train /= 128.
X_test /= 128.

In [6]:

dataset_train = tf.data.Dataset.from_tensor_slices((X_train, Y_train))

dataset_train = dataset_train.shuffle(buffer_size=100).batch(bs).prefetch(50)

In [7]:
dataset_test = tf.data.Dataset.from_tensor_slices((X_test, Y_test))

dataset_test = dataset_test.batch(bs).prefetch(50)

In [8]:
model = resnet.ResNet.build(64, 64, 3, nb_classes, (3, 4, 6),
		(64, 128, 256, 512), reg = 0.0005, dataset = "tiny_imagenet")
opt = SGD(lr = 1e-1, momentum = 0.9, nesterov = True)

model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics = ["accuracy"])

C:\Users\DAVID\anaconda3\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [9]:
epochs = 20

history = model.fit(dataset_train, batch_size=bs, epochs=epochs, validation_data=dataset_test, callbacks=[model_checkpoint])

Epoch 1/20
782/782 [==============================] - ETA: 0s - loss: 4.8963 - accuracy: 0.1036
Epoch 00001: val_accuracy improved from -inf to 0.12680, saving model to models\model
782/782 [==============================] - 3518s 4s/step - loss: 4.8963 - accuracy: 0.1036 - val_loss: 4.6566 - val_accuracy: 0.1268
Epoch 2/20
782/782 [==============================] - ETA: 0s - loss: 4.0420 - accuracy: 0.2168
Epoch 00002: val_accuracy improved from 0.12680 to 0.20040, saving model to models\model
782/782 [==============================] - 3535s 5s/step - loss: 4.0420 - accuracy: 0.2168 - val_loss: 4.2146 - val_accuracy: 0.2004
Epoch 3/20
782/782 [==============================] - ETA: 0s - loss: 3.7016 - accuracy: 0.2769
Epoch 00003: val_accuracy improved from 0.20040 to 0.23370, saving model to models\model
782/782 [==============================] - 3510s 4s/step - loss: 3.7016 - accuracy: 0.2769 - val_loss: 4.0280 - val_accuracy: 0.2337
Epoch 4/20
782/782 [=============================

In [10]:
epochs_passed = 20
model.save_weights( 'models/model_weights_Img_aug_after_%s_epoch.h5' % epochs_passed )
model.save( 'models/custom_resent50_model.h5' )

C:\Users\DAVID\anaconda3\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [ ]:
from keras.models import load_model
epochs_passed = 20

# Returns a compiled model identical to the previous one
model = load_model( 'models/custom_resent50_model.h5')
model.load_weights( 'models/model_weights_Img_aug_after_%s_epoch.h5' % epochs_passed )

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

nb_epoch = 10 # epoch 21-30
# Fit the model on the batches generated by datagen.flow().